### Import Libraries

In [1]:
import json
import tweepy
import pandas as pd
import numpy as np
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### Set up Twitter API

In [2]:
consumer_key = 'pJwUhqD8L9wlljqHOEJj1Egis'
consumer_secret = 'wS5dUxVpcFW1dMublVqjnfswo9zfNwO6UELzXlFca8MSkXhYoN'
access_token = '1067451890559000577-PjZTRWcZhRr89DALnNBd1lZlKS82ej'
access_secret = '4VZy0Z7kKH71pt0H7b99tH7iQECEjd5N0MbTIXCCa1MBW'
# Mehdi's access tokens
#consumer_key = 'Yjab88aGQENfS8iOEkuWl2ib9'
#consumer_secret = 'LWjZhWE58Lcv048GEfwJHSp5sxzXJzfirc7YNblMrlDDf25vv4'
#access_token = '383323140-GsuErAojAYG5FnbSfJ6OV2Ts0Z5VYUju06Ca474Z'
#access_secret = 'VfsaYnW8mgmrwDEbKGkZ7f6OyUW1XtU4FjKIdTcDSg2QE'
#Pass our consumer key and consumer secret to Tweepy's user authentication handler
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

#Pass our access token and access secret to Tweepy's user authentication handler
auth.set_access_token(access_token, access_secret)

#Creating a twitter API wrapper using tweepy
api = tweepy.API(auth)

#Error handling
if (not api):
    print ("Problem connecting to API")

### Number of requests left

In [3]:
#Switching to application authentication
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

#Setting up new api wrapper, using authentication only
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
 
#Error handling
if (not api):
    print ("Problem Connecting to API")
    
#You can check how many queries you have left using rate_limit_status() method
api.rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 450, 'remaining': 450, 'reset': 1554847791}}

In [6]:
counter = 0
for page in tweepy.Cursor(api.user_timeline,screen_name='UNDP', lang = 'en').pages(2):
    counter += 1 
    print(page)

[Status(_api=<tweepy.api.API object at 0x0000024B4FB4F1D0>, _json={'created_at': 'Tue Apr 09 20:27:18 +0000 2019', 'id': 1115712813291458561, 'id_str': '1115712813291458561', 'text': "135 social impact bonds are already helping 28 countries achieve #SDGimpact. We're exploring the feasibility of… https://t.co/WRo8y5wqeK", 'truncated': True, 'entities': {'hashtags': [{'text': 'SDGimpact', 'indices': [65, 75]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/WRo8y5wqeK', 'expanded_url': 'https://twitter.com/i/web/status/1115712813291458561', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [113, 136]}]}, 'source': '<a href="https://www.hootsuite.com" rel="nofollow">Hootsuite Inc.</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 20596281, 'id_str': '20596281', 'name': 'UN Development', 'screen_name': 'UNDP', 'location': '170 countr

[Status(_api=<tweepy.api.API object at 0x0000024B4FB4F1D0>, _json={'created_at': 'Mon Apr 08 18:40:14 +0000 2019', 'id': 1115323483389558784, 'id_str': '1115323483389558784', 'text': 'RT @UNYouthEnvoy: Young inspiring leaders @NouraBerrouba @eyp_int &amp; Yolanda Joab of Island PRIDE joined Kasunjith Satanarachchi @UNVoluntee…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'UNYouthEnvoy', 'name': 'UN Youth Envoy', 'id': 2923724679, 'id_str': '2923724679', 'indices': [3, 16]}, {'screen_name': 'NouraBerrouba', 'name': 'Noura Berrouba', 'id': 21706658, 'id_str': '21706658', 'indices': [42, 56]}, {'screen_name': 'eyp_int', 'name': 'European Youth Parliament', 'id': 94938275, 'id_str': '94938275', 'indices': [57, 65]}], 'urls': []}, 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, '

### Creating tweets dataframe
#### Initializing lists

In [7]:
tweets = []
likes = []
retweets = []
replies = []
country = []
coordinates = []
followers = []
posts = []

#### Query creation via keywords

In [10]:
most_common_politics = ['presid', 'minist', 'elect', 'prime', 'govern', 'leader', 'obama', 'trump', 'nation', 'state', 'party', 'barack', 'polit', 'talk', 'president', 'donald', 'vote', 'opposit', 'pm', 'offici', 'deal', 'polic', 'power', 'poll', 'parliament', 'rule', 'support', 'face', 'republican', 'warn', 'gener', 'peac', 'accus', 'claim', 'crisi', 'campaign', 'white', 'democrat', 'announc', 'trade', 'putin', 'chief', 'aid', 'candid', 'urg', 'win', 'resign', 'ban', 'death', 'senat', 'mugab', 'sign', 'summit', 'clinton', 'protest', 'investig', 'terror', 'diplomat', 'lead', 'independ', 'corrupt', 'mp', 'washington', 'secretar',  'victory','victories', 'fire', 'kill', 'decis', 'policy','policies', 'vladimir', 'democrac', 'sanction', 'result', 'politician',  'threat', 'debat', 'controvers', 'cabinet', 'govt', 'speech', 'agreement', 'affair', 'term', 'declar', 'congress', 'rival', 'council', 'activist', 'reform', 'voter', 'rall', 'address', 'ambassador', 'relat', 'conserv', 'tie', 'parliamentar', 'coalit', 'appeal', 'negoti', 'condemn',  'propos', 'governor', 'referendum',  'accord', 'conflict', 'leadership', 'justic',  'king', 'deput', 'inquir','riot', 'ally','allies', 'cooper', 'movement', 'spark', 'forum', 'embass', 'nato', 'repres', 'prosecutor', 'republ', 'elector', 'unrest','deport', 'spokesman',  'regim', 'mayor', 'attorney', 'nomine', 'demonstr', 'kim',  'communist', 'ministr', 'treaty','treaties', 'counterpart', 'statement', 'commission', 'brexit', 'eurozon', 'assembl', 'royal', 'merkel', 'dictat', 'queen', 'kremlin', 'gaza', 'pope', 'peacekeep', 'chairman', 'princ', 'revolut', 'kingdom', 'pentagon','freedom','expression','anti', 'offic', 'resist','deploy','crackdown','econom','legisl','negoci','renogoci','march','right','left','wing','censor','censorship','author']
most_common_climate = ['earthquak', 'flood', 'hurrican', 'storm', 'quak', 'kill', 'tsunami', 'damag', 'tornado', 'wildfir', 'disast', 'typhoon', 'devast', 'evacu', 'volcano', 'death', 'warn', 'aid', 'dead', 'magnitud', 'weather', 'help', 'survivor', 'crisi', 'climat', 'govern', 'erupt', 'forc', 'fear', 'threaten',  'victim', 'struck', 'emerg', 'rescu', 'fire', 'risk', 'face', 'unit', 'fukushima', 'deadly',  'radiat', 'cyclon',  'weatherwatch',  'landslid', 'destroy', 'trigger', 'injur', 'flee', 'struggl', 'firefight', 'eyewit',  'die', 'stricken', 'crippl', 'respons', 'threat', 'extrem', 'lava', 'safet', 'destruct', 'bushfir', 'recover', 'impact',  'strike',  'rescuer', 'alert', 'rubbl', 'blaze', 'drought', 'disrupt', 'ravag', 'collaps', 'aftershock', 'aftermath', 'danger', 'avalanch', 'mudslid', 'catastroph', 'landfal', 'heatwav', 'shelter', 'homeless', 'injur', 'remot', 'volcan',  'bury','buri', 'strand', 'torrenti', 'surviv', 'rip', 'chao', 'concern', 'floodwat', 'explos', 'suffer', 'lost', 'spark', 'ruin', 'escap',  'humanitarian',  'safe', 'superstorm', 'panic', 'thunderstorm', 'evacue', 'chernobyl', 'epidem', 'inund', 'flame', 'delug', 'traged', 'snowstorm', 'hazard']
most_common_health = ['health', 'drug', 'cancer', 'risk', 'patient', 'diseas', 'ebola', 'food', 'treatment', 'heart', 'viru',
                      'medic', 'healthcar', 'hospit', 'doctor', 'fda', 'vaccin', 'death', 'flu', 'blood', 'treat', 'drugmak',
                      'infect', 'medicin', 'diabet', 'pharmaceut', 'zika', 'die', 'clinic', 'breast', 'smoke', 'therap',
                      'surger', 'organ', 'abort', 'lung', 'aid', 'novart', 'roch', 'hiv', 'sanof', 'brain', 'cigarett',
                      'older', 'drink', 'obes', 'johnson', 'pfizer', 'chang', 'tobacco', 'merck', 'astrazeneca', 'pill', 
                      'medicar', 'ill', 'cell', 'exercis', 'patent', 'epidem', 'opioid', 'hepat', 'surviv', 'cholesterol',
                      'pharma', 'pregnanc', 'born', 'sleep', 'mental', 'diet', 'eye', 'symptom', 'chronic', 'glaxosmithklin',
                      'antibiot', 'gene', 'nurs', 'inject', 'head', 'diagnos', 'injur', 'marijuana', 'biotech', 'suicid',
                      'medicaid', 'fever', 'skin', 'sexual', 'order', 'prostat', 'syndrom', 'alzheim', 'smoker', 'health',
                      'pregnant', 'respirator', 'mosquito', 'therapeut', 'fat', 'infant', 'elderl', 'kavanaugh', 'probe',
                      'pharmac', 'kidney', 'liver', 'abus', 'sugar', 'cdc', 'alcohol', 'contracept', 'insulin', 'coly',
                      'addict', 'asthma', 'amgen', 'vitamin', 'bacteria', 'stress', 'arthriti', 'physic', 'dose', 'dengu',
                      'scleros', 'immun', 'gsk', 'allerg', 'allergan', 'immunotherap', 'malaria', 'mylan', 'swine',
                      'cure', 'addit', 'tumor', 'sick', 'cholera', 'dementia', 'colon', 'disabl', 'surgeon', 'biosimilar',
                      'fatal', 'physician', 'superbug', 'leukemia', 'fresh', 'adolesc', 'transplant', 'overweight', 'concuss',
                      'biogen', 'bone', 'fruit', 'menopaus', 'chemotherap', 'migrain', 'autism', 'illeg', 'regeneron', 'taint',
                      'hurt', 'green', 'calori', 'nuclear', 'recov', 'nicotin', 'pathogen', 'steroid', 'hormon', 'pediatrician',
                      'anxiet', 'salmonella', 'organis', 'habit', 'tuberculos', 'inhal', 'cardiovascular', 'squibb',
                      'listeria', 'psorias', 'protein', 'dna', 'antidepress', 'avastin', 'diagnos', 'fertil', 'rheumatoid',
                      'cv', 'diagnost', 'worsen', 'radiat', 'melanoma', 'statin', 'soda', 'ovarian', 'bowel', 'cardiac', 'knee',
                      'biotechnolog', 'arter', 'newborn', 'dead', 'bladder', 'oncolog', 'breath', 'arm', 'adhd', 'healthier',
                      'olymp', 'cough', 'surgic', 'sugari', 'nutrit', 'polio', 'anthrax', 'poison', 'microcephal']
most_common_war = ['protest', 'kill', 'forc', 'attack', 'polic', 'clash', 'milit', 'soldier', 'militar', 'fighter', 'arm',
                   'injur', 'secur', 'arrest', 'airstrik', 'houthi', 'rebel', 'bomb', 'shot', 'strike', 'syrian', 'raid',
                   'taliban', 'saudi', 'militia', 'fight', 'destroy', 'warplan', 'artiller', 'gunmen', 'afghan', 'troop',
                   'violenc', 'burn', 'riot', 'damascu', 'battl', 'grenad', 'weapon', 'gun', 'assault', 'violent', 'rocket',
                   'missil', 'ambush', 'guard', 'rioter', 'myanmar', 'spla', 'convoy', 'amisom', 'loot', 'pakistan', 'gunfir',
                   'murder', 'kidnap', 'abduct', 'bjp', 'bombard', 'blast', 'bullet', 'rape', 'fardc', 'destruct', 'patrol',
                   'shoot', 'tactic', 'bomber', 'mob', 'kia', 'tahrir', 'tortur', 'hurl', 'khairpur', 'defenc', 'withdrew',
                   'ghazni', 'sadah', 'insurg', 'assassin', 'fought', 'beaten', 'war', 'battalion', 'jamaat', 'hartal',
                   'peshawar', 'larkana', 'pelt', 'vandal', 'puntland', 'sniper', 'combat', 'quetta', 'islami', 'najran',
                   'rout', 'farah', 'faryab', 'dalit', 'muhammad', 'afrin', 'awami', 'hostag', 'tnla', 'polic', 'ugandan',
                   'balaka', 'hudaydah', 'saf', 'terror', 'jaysh', 'chandigarh', 'bab', 'bayda', 'defens', 'fnl', 'herder', 
                   'marib', 'fdlr', 'pistol', 'urozgan', 'ammunit', 'penh', 'aden', 'ngo', 'adf', 'surrend', 'crime',
                   'nigerian', 'nagar', 'threaten', 'gang', 'freedom', 'conflict', 'kunduz', 'haryana', 'balochistan',
                   'ahrar', 'baton', 'duma', 'harass', 'asir', 'imbonerakur', 'ludhiana', 'aircraft', 'gunshot', 'ghouta',
                   'splm', 'evict', 'oromo', 'bujumbura', 'baidoa', 'sukkur', 'chittagong', 'infantri', 'karnataka', 'manipur',
                   'shahe', 'taiz', 'reveng', 'ranger', 'paktia', 'updf', 'raider']
most_common_ = list(set(most_common_politics +most_common_climate+most_common_health+most_common_war))
query = str()
for i in most_common_:
    query += i + ' OR '
    if len(query)>400:
        break
query = query[:-4]

#### Request to Twitter API

In [11]:
counter = 0
for tweet in tweepy.Cursor(api.search,q=query, lang = 'en',  tweet_mode='extended').items(1000):        
        if 'retweeted_status' not in tweet._json.keys():
            counter += 1
            #print(counter, '|', tweet._json['full_text'])
            #print('-')
            tweets.append(tweet._json['full_text'])
            likes.append(tweet.favorite_count)
            retweets.append(tweet.retweet_count)
            try:
                replies.append(tweet.reply_count)
            except:
                replies.append(np.nan)
            try:
                country.append(tweet.place.country)
            except:
                country.append(np.nan)
            try:
                coordinates.append(tweet.coordinates.coordinates)
            except:
                coordinates.append(np.nan)
            try: 
                followers.append(tweet.user.followers_count)
            except:
                followers.append(np.nan)
            try:
                posts.append(tweet.user.statuses_count)
            except:
                posts.append(np.nan)

print(counter)   

261


#### Create DataFrame

In [12]:
df = pd.DataFrame({'Tweets':tweets,'Likes':likes,'Retweets':retweets,'Replies':replies,'Country':country,'Coordinates':coordinates,
                  'Followers':followers,'Posts':posts})

In [13]:
df

,Tweets,Likes,Retweets,Replies,Country,Coordinates,Followers,Posts
0,RT Cryptojii_HQ: One of the cryptographers most often cited in #Satoshi Nakamoto's white paper has reason to think the #crypto industry's future is bright. coindesk https://t.co/mn9gAjb8TZ,0,0,NaN,NaN,NaN,66,9156
1,@PiyushRaiTOI @Joydas Apologists activated already ?\nThe pic is from the home of 1 alleged terrorist\nA video has been seized by NIA where an alleged terrorist is telling how to assemble bomb using a circuit\n\nhttps://t.co/07cIEOBO0o\n\nA so called journey shud do basic research at least\n#IndiaAgainstISIS,0,0,NaN,NaN,NaN,62,5124
2,"@tedlieu @MattWhitaker46 @POTUS Attack attack attack! THAT'S all you care about. Who was this guy to you 2 months ago? Now he's in trouble, WHY? Cuz OUR PRESIDENT appointed him to work for us. Now attack attack attack! \nWe see you &amp; we know!!",0,0,NaN,NaN,NaN,225,7656
3,CM Sindh SYED @MuradAliShahPPP and @murtazawahab1 chairs Ijlas LAW &amp; ORDER ..SOT\n(CHIEF MINISTER HOUSE SINDH)\n@SaeedGhani1 @WaqarMehdiPPP1 @RashidRabbani14 https://t.co/0cgjdIHRLF,0,0,NaN,NaN,NaN,12244,16940
4,Tell me; are you just a parrot for Fox news and Trump?? Do you actually have a brain that you use? You sound exactly like the lunatic in the White House spreading lies about these people. Karma is coming and she’s a bitch.#ResignTrump https://t.co/MXfNC87ODS,0,0,NaN,United States,NaN,3015,37274
5,I liked a @YouTube video https://t.co/9S0YpNl1Tu I have a new idea! - White Rider build series #64,0,0,NaN,NaN,NaN,99,3035
6,DRINK https://t.co/E3AvaiiaPx,0,0,NaN,NaN,NaN,157,19626
7,do i look like a fucking loonatic? of course i would support them https://t.co/QRUldWU9MB,0,0,NaN,NaN,NaN,41,176
8,"Your support is critical to protecting views that never get old. Donate to OSI to protect land for people, for habitat, forever. https://t.co/WLwfD1W8Ug https://t.co/1pJ4FBhXfH",0,0,NaN,NaN,NaN,1378,3048
9,They voted for a senate and president that does. https://t.co/kDNaqNjzns,0,0,NaN,NaN,NaN,1799,82871


In [ ]:
relevant = [0,1,1,0,1,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0]

#### Dropping 0 followers and creating posts/followers ratio

In [48]:
df = df[df['Followers'] != 0]

In [49]:
df['Posts/Followers'] = df.apply(lambda x : x['Posts']/x['Followers'],axis = 1)

In [50]:
df['Index'] = [i for i in range(df.shape[0])]
df = df.set_index('Index')

### Filter to keep only 'relevant' tweets

In [51]:
threshold = 0.25
indexes = []
post_follower_ratio = 30
non_found = 0
for i in df.index:
    counter = 0
    text = df.loc[i,'Tweets']
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',' ', text)
    text = re.sub('(@[A-Za-z0-9]+)', ' ', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.split(' ')
    text = list(filter(lambda a: a != '', text))
    if len(text) >= 20:
        if df.loc[i,'Posts/Followers'] <= post_follower_ratio:
            text = ' '.join(text).lower().split()
            ps = PorterStemmer()
            text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
            for k in most_common_:
                if k in text:
                    counter += 1
            if (counter/len(text)) < threshold:
                non_found += 1
            else:
                #print(i,round(100*counter/len(text),2),'%', '|',df.loc[i,'Tweets'])
                #print('-')
                indexes.append(i)
    else: 
        print(df.loc[i,'Tweets'])
        print('_')



@BecksOfBucks As deadly as cancer. They both are heartless and exist only to destroy.
_
✨ Sign Up For FREE Access to ✨

Eileen's Latest Podcasts on highly sensitive people and empaths

https://t.co/hCYdpvZcua #sensitivesouls #hsp https://t.co/aMGHibss0F
_
@AnneTibbets Absolutely. It can melt ones brain if a person goes nonstop.
_
The Man Who Rigged The World Series: The Making of the Mob: New York https://t.co/9B4F04nKri via @YouTube
_
Patience mat turn into a patient!! Cats have claws https://t.co/uB5mGh63qr
_
Russia says its Su-57 stealth fighter will be armed with deadly hypersonic missiles that can defeat all US defences
https://t.co/mlJD5Up0qH
_
Black Infant Death Rates Down In Sacramento Following Massive Community Efforts https://t.co/4qSLX4bAYM
_
Want to work at Worthington? We're #hiring in #Theodore, AL! Click for details: https://t.co/zbKEOhanB7 #internship #Job #Jobs #CareerArc
_
@ngb_lilbreezy now when he shoot up the next mini mall yall still gonna laugh?
_
in particular 

@NinjahTV @Nadeshot @100Thieves Shoot you know I was but then it was gone ya know
_
Enjoy Shopping at Alex &amp; Ani AND Supporting the Work of SpeakOUT! - https://t.co/rdvyMWgLzP
_
@Simon4NDorset @annofarabia More lies from an MP 🤔
_
@anthonyfurey Are @Twitter and @jack going to deliver the death penalty as well?
_
@dylanakent Maybe they should brandish those deadly rulers... https://t.co/tjikHlhjCz
_
Y’all still use iMessage games as a way to shoot shots?
_
अनुराग भदौरिया 

BAN THIS MOTHERXXXXD OF NAMAZ WAADI ON TV DEBATES NOW
_
Not enough black people in Baton Rouge jury pools, lawyers for man accused in son's beating death say https://t.co/J4DiC5DWnB
_
अनुराग भदौरिया 

BAN THIS MOTHERXXXXD OF NAMAZ WAADI ON TV DEBATES NOW https://t.co/t5I3cxp4ry
_
The Nazi leader, Hitler, from 1942 until his death in 1945, was addicted to crystal meth.
_
i miss death note
_
@hotskillet @aravosis I can feel my brain cells dying in proximity 😳
_
@miss_rissaroo My death
_
Insights into the intrinsic c

Watch Stunning Video Showing Russian Rocket Leaving Earth, As Seen by the ISS — Curiosmos https://t.co/kp68NtGh5o
_
@DionDublinsDube @sterling7 Simple just ban for life
20 sky camera's all over the stadium his ugly mug surely on one
_
F*CK THIS M.BISON FIGHT!! &gt;:U #SmashBrosUltimate https://t.co/dOTcXuT2ie
_
Me: ok, I'm fine, everything's good
Me: let's listen to The Cure
_
Ban all refs wtf are we doing
_
@fakeyfan uR VERY WELCOME thank you for blessing my brain
_
Refugee beats praying priest to death with metal cross in Germany https://t.co/YeicbwTpXE
_
@BecksOfBucks As deadly as cancer. They both are heartless and exist only to destroy.
_
✨ Sign Up For FREE Access to ✨

Eileen's Latest Podcasts on highly sensitive people and empaths

https://t.co/hCYdpvZcua #sensitivesouls #hsp https://t.co/aMGHibss0F
_
@AnneTibbets Absolutely. It can melt ones brain if a person goes nonstop.
_
The Man Who Rigged The World Series: The Making of the Mob: New York https://t.co/9B4F04nKri via @YouTub

@sortius There is no such thing as a perfect partner and if there were you’d be bored to death
_
@discordapp @LogitechG Nobody can ever think of/create another color that is not on the color spectrum.
_
[Daily Mail] - Norwich 3-2 Bolton: Championship leaders snatch victory at the death | SportSpyder https://t.co/TeUSLgQu2x
_
New on ebay: MP Select ABS Plus+ Premium 3D Filament  1kg 3mm  White 
 https://t.co/PlA8iUgb55 https://t.co/3w2OgBDQwa
_
Landmark 5G Criminal Trial, "It's a Weapon System", MI6 Concerned https://t.co/bBPQovPf7s via @YouTube
_
@THubbmusic  you want to ban guns, I'm banning you instead.
_
this boy asked me would I shoot him for 1mil , why I tell him “in a heartbeat”
_
@Lord_Sugar @Joyceeee77 How did you manage to get where you are with so little brain?
_
@TheFinesseKing3 Bro on god it could easily be a life or death situation when them eyes open back up
_
School has destroyed my brain
_
Another first for me today... Sat talking to a patient and their dog starts to sh

i’m all stressed pls why does my brain like to remember things i didn’t ask for
_
Need more reasons to play? It’s brain building! https://t.co/8iY5DOtjS8
_
when i finally design this Big Sexe death metal god minotaur its over for you hoes
_
@WeeCarters Thanks for being so patient :3
_
@jrosario319 Ben simmons even better than john wall and that nigga cant shoot😴
_
Sheez. 3 years from now. Iz*one is already disbanded. Brain remove this thought. https://t.co/KNyk1UxsNN
_
@WayneDupreeShow I heard @NancyPelosi say  three COATS, not coasts . Dentures mAlfunction or her Brain is softening? #NancyPelosi
_
2 things you can be sure of in life....death. And a band not showing up on time
_
Duh! So, we're asking you now, what are some of your favorite lines that this warlock brain produced?
_
UNIVERSE BRAIN THOUGHT
WAYNE IS A CATBOY AND THAT'S WHY A LOT OF CATS LOOK LIKE HIM
_
I think I’m booping @apieceofbread to death
_
I added a video to a @YouTube playlist https://t.co/iIV6bYZ8rD i-pulse_Lung 

Once a patient has registered for the card, the entire medical history along with the credentials will be recorded.

https://t.co/dDq1IijipA
_
The kid had a galaxy brain moment.
_
Apparently if you show your butt on social media you “have no brain”. I hate people sometimes
_
If you get booked for simulation you should get a one match ban simple as that  #ARSHUD
_
OMG! Have any leftists cried about French police using "chemical weapon" and committing war crimes yet? #LeftistsFakeOutrage 🙄 https://t.co/nYIb25gPLS
_
We're looking for a community driven Verizon Sales Representative in #Pascagoula, MS - know anyone? https://t.co/8yNgcJIjK2 #Sales #CareerArc
_
@3shells1991 @BTS_twt They are preparing for our death i am sooo scared
_
@3winters3 I disagree, officer Bennett saved me from an angry mob of idiots
_
Got a photo shoot in 20 min I should prooooooobably get ready for
_
Did a shoot with my best mate today https://t.co/c9nH7Uu5jG
_
Preview from today's shoot :) https://t.co/mrBmU60cUy
_

Licence To Kill. Wren, Bullfinch, Skylark, Oyster Catcher, Robin.....40 Species On Natural England's Death List - Jason Endfield https://t.co/PghkbjVOTV
_
There's an insect inside my brain
_
#NowPlaying Funk5um1 - A New Journey 20 (Original Mix) on https://t.co/xpMYYHBSi5 #DjMixes99radeo.com #EDM #HOUSE #TECHNO #RADIO
_
When you shoot your shot and it goes exactly as planned https://t.co/0povltVpMP
_
@BCusterTV @Flemings Beautiful family Brain
_
US appeals court refuses to allow Trump asylum ban https://t.co/76GBa0agS4
_
Watch Stunning Video Showing Russian Rocket Leaving Earth, As Seen by the ISS — Curiosmos https://t.co/kp68NtGh5o
_
@DionDublinsDube @sterling7 Simple just ban for life
20 sky camera's all over the stadium his ugly mug surely on one
_
F*CK THIS M.BISON FIGHT!! &gt;:U #SmashBrosUltimate https://t.co/dOTcXuT2ie
_
Me: ok, I'm fine, everything's good
Me: let's listen to The Cure
_
Ban all refs wtf are we doing
_
@gwslrd @realDonaldTrump French people love trump and poutine

Once a patient has registered for the card, the entire medical history along with the credentials will be recorded.

https://t.co/dDq1IijipA
_
The kid had a galaxy brain moment.
_
Apparently if you show your butt on social media you “have no brain”. I hate people sometimes
_
If you get booked for simulation you should get a one match ban simple as that  #ARSHUD
_
OMG! Have any leftists cried about French police using "chemical weapon" and committing war crimes yet? #LeftistsFakeOutrage 🙄 https://t.co/nYIb25gPLS
_
We're looking for a community driven Verizon Sales Representative in #Pascagoula, MS - know anyone? https://t.co/8yNgcJIjK2 #Sales #CareerArc
_
@3shells1991 @BTS_twt They are preparing for our death i am sooo scared
_
@3winters3 I disagree, officer Bennett saved me from an angry mob of idiots
_
Got a photo shoot in 20 min I should prooooooobably get ready for
_
Did a shoot with my best mate today https://t.co/c9nH7Uu5jG
_
Preview from today's shoot :) https://t.co/mrBmU60cUy
_

thumba-thena. your presence is required it’s life or DEATH!
_
WOW...FRANCE! SUCH A LOVELY PLACE TO LIVE NOWADAYS... https://t.co/KUtZnD1WhR
_
Goodie Mob featuring Outkast | "Black Ice" (1998) https://t.co/xF6rqvMaX0 via @YouTube
_
pain muffled brain muzzled
_
@IamTheToph @cure_magazine I'll be thinking of you, let me know if I can support you in any way!
_
@ColmDaly Tbh my brain is absolutely Milton Friedman #Economics
_
think with your brain not your heart
_
i probably bored them to death but i dont CARE?? how could u say smth so..... sad
_
🐒🌴 Save the orangutan. Keep the palm oil ban. Sign the petition today. @BestForBritain https://t.co/7oz75lTjoS
_
Agony and Death at Washington University: A Whistleblower Case https://t.co/LXT1XeqcC5
_
@morningbrings Perfect. I’ll shoot you a message tomorrow, then?
_
someone please shoot me
_
Day 9 of 12 Days of Christmas Sales! 20% OFF continues for one final day! #GiftsForHim #MerryChristmas #MensSwimwear - https://t.co/511fzRQJ4p https://t.co/s

Father denied visa for slain daughter's funeral as suspect charged with rape, murder https://t.co/A2FCO3CMwO via @usatoday
_
New York Times: Kushner offered advice to Saudi crown prince after journalist's death - CNN Politics https://t.co/mtIRI3ENzG
_
@Ivorybluemv That's not good, it felt like ages to get our ban removed, it's a pain...
_
@sortius There is no such thing as a perfect partner and if there were you’d be bored to death
_
@discordapp @LogitechG Nobody can ever think of/create another color that is not on the color spectrum.
_
[Daily Mail] - Norwich 3-2 Bolton: Championship leaders snatch victory at the death | SportSpyder https://t.co/TeUSLgQu2x
_
New on ebay: MP Select ABS Plus+ Premium 3D Filament  1kg 3mm  White 
 https://t.co/PlA8iUgb55 https://t.co/3w2OgBDQwa
_
Landmark 5G Criminal Trial, "It's a Weapon System", MI6 Concerned https://t.co/bBPQovPf7s via @YouTube
_
@THubbmusic  you want to ban guns, I'm banning you instead.
_
this boy asked me would I shoot him for 1mil

### Create relevant tweet dataframe

In [52]:
df_relevant = df.ix[indexes]
#df.loc[indexes,:]

C:\Users\antoi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [53]:
df_relevant = df_relevant.drop_duplicates()

In [54]:
df_relevant['Index'] = [i for i in range(df_relevant.shape[0])]
df_relevant = df_relevant.set_index('Index')

In [55]:
df_relevant

,Tweets,Likes,Retweets,Replies,Country,Coordinates,Followers,Posts,Posts/Followers
Index,,,,,,,,,
0,@andygrayhawkes @realDonaldTrump There actually throwing big ass rocks which can be deadly force one hit in the head can cause brain bleeds hematoma’s and it will kill a person or put them in a vegetable state a coma but if we shoot at them or try to stop them were wrong,0,0,NaN,NaN,NaN,1,16,16.000000
1,"@freepearn @stvstheworld @gremisch @DigbyReturns given a graph of all leo's organs, which algorithm will help you find the shortest path between his left lung and right kidney?",2,0,NaN,NaN,NaN,2898,19873,6.857488
2,"1/2: Tonight, our officers attended a #Domestic disturbance. A male involved became aggressive and resisted arrest, using a weapon to assault colleagues trying to #protect those at harm. He was subsequently detained, and one of our injured colleagues was taken to hospital... https://t.co/6mI3ehIjbp",0,0,NaN,NaN,NaN,1551,880,0.567376
3,@dbongino Democrats are the party of drugs &amp; alcohol there are no brain cells left that is why they have no ideas and cannot act in civilized manner,0,0,NaN,NaN,NaN,1905,14575,7.650919
4,"@lammotivational @emendes14 @MichelleObama 1/Part of the problem in this country, is that when politicians die, they become something in death that they never were in life.When John McCain ran for president against Obama he was “an unstable racist unfit to lead”. In death, he became “An American Icon”. How does that work?",0,0,NaN,NaN,NaN,1560,9487,6.081410
5,"You can use spirituality to support conventional and proven effective treatments, but you can't use spirituality only to ""cure"" it.",0,0,NaN,NaN,NaN,2428,27461,11.310132
6,@siano2020 Working in the healthcare field has taught me a lot and one of those is there are things worse than death. Anyone who has to face their last days with terminal illnesses deserve the right to make the decision to go out as they choose.,0,0,NaN,NaN,NaN,1606,5599,3.486301
7,"you know what’s in my goddamn head right now at 11:30pm?\n\nare we gonna have a problem? \nyou got a bone to pick?\nyou’ve come so far,\nwhy now are you pulling on my dick?\n\n....\n\ni literALLY haven’t listened to this soundtrack in MONTHS why is my brain like this",0,0,NaN,NaN,NaN,200,2858,14.290000
8,"Hey remember when we tear gassed people on the border (just like Obama did, this isn't new to Trump) and everyone was calling it a war crime and a chemical weapon?\n\nGuess what France is doing RIGHT NOW to their own citizens. Why aren't the same people crying foul?",0,0,NaN,NaN,NaN,396,4634,11.702020


### Manualy categorize the tweets

In [83]:
Cat = ['Politics','Health','War and Conflict','Politics','Politics','Health','Health','Politics','War and Conflict','Politics',
      'Politics','Health','War and Conflict','Politics','Politics','Politics','Politics','Health','Politics','War and Conflict','Politics',
      'Politics','Health','Politics','War and Conflict','Health','War and Conflict','Health','Politics','War and Conflict',
      'War and Conflict']

In [84]:
df_relevant['Category'] = Cat

### Add a url column

In [85]:
df_relevant['URL_in_tweet'] = df_relevant['Tweets'].apply(lambda x : re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', x))

### Save the data

In [86]:
df_relevant.to_csv('C://users/antoi/desktop/data_UN/tweet_api/Crimson_12_08_2018_cat1.csv')